this will examine the distribution with respect to eg rho, conditional on the eg rho of the previous generation

In [ ]:
%matplotlib inline

# a lot of this is hack and pasted from jup_plot_logs
# but prefer to hack it here, than fill jup_plot_logs with random specific stuff

import matplotlib.pyplot as plt
import math, glob, json, os, subprocess, time, datetime, sys, importlib, math
from os import path
from os.path import join
from collections import defaultdict, OrderedDict
import numpy as np
import scipy.stats
sys.path.append('..')

import ulfs
from ulfs.params import Params
from ulfs import graphing, graphing_common, graphing_indexes
importlib.reload(ulfs.graphing)
importlib.reload(ulfs.graphing_common)
importlib.reload(ulfs.graphing_indexes)

log_dir = '../logs'

def run_graphs(script, refs, prev_measure, next_measure, add_linear=False):
    prev_v_l = []
    next_v_l = []

    comms_dropout = None
    dropout = None

    for ref in refs:
        e2e_logfiles = glob.glob(f'{log_dir}/log_{script}_{ref}_*.log')
        e2e_logfiles.sort()
    
        for i, file in enumerate(e2e_logfiles):
            log_filepath = join(log_dir, file)
            num_lines = graphing_common.get_num_lines(log_filepath)
            if num_lines < 2:
                continue
            meta = graphing_common.read_meta(log_filepath)
            meta['num_lines'] = num_lines
            argv = meta['argv']
            params = Params(meta['params'])
            if params.ref != ref:
                continue
            meta['log_filepath'] = log_filepath
            if comms_dropout is None:
                comms_dropout = params.__dict__.get('comms_dropout', 0)
                dropout = params.dropout
            else:
                assert dropout == params.dropout
                assert comms_dropout == params.__dict__.get('comms_dropout', 0)
            meanings = f'{params.num_meaning_types}x{params.meanings_per_type}'
            rmlm = f'{params.model}_{params.link}_{meanings}'
            exclude = False
            for filter in filters:
                if filter not in rmlm:
                    exclude = True
            if exclude:
                print('excluded')
                continue
            prev_v = None
            with open(log_filepath, 'r') as f_in:
                for n, line in enumerate(f_in):
                    if n == 0:
                        continue
                    d = json.loads(line)
                    if d.get('record_type', 'ilm') != 'ilm':
                        continue
                    episode = d['episode']
                    next_v = d[next_measure]
                    new_prev_v = d[prev_measure]
                    if prev_v is not None:
                        prev_v_l.append(prev_v)
                        next_v_l.append(next_v)
                    prev_v = new_prev_v
    
    plt.xlabel('previous generation ' + prev_measure)
    plt.ylabel('this generation ' + next_measure)
    plt.title(','.join(refs))
    plt.scatter(prev_v_l, next_v_l)
    if prev_measure == next_measure and add_linear:
        plt.plot(prev_v_l, prev_v_l)
    
    rho, p = scipy.stats.spearmanr(a=prev_v_l, b=next_v_l)
    return rho

value_keys = ['e2e_acc', 'rho', 'holdout_acc']
target_episode = 0

filters = []
max_step = 25000
units = ''
num_rows = 2

# refs = ['ilmb67', 'ilmb68', 'ilmb69']
# refs = ['ilmc128']

def diag_plots(refs):
    scores = []
    plt.figure(figsize=(20, 3.5))
    for i, measure in enumerate(['e2e_acc', 'holdout_acc', 'rho']):
        plt.subplot(1, 4, i + 1)
        rho = run_graphs(script='ilm_rnn2018janb', refs=refs,
                   prev_measure=measure, next_measure=measure, add_linear=True)
        scores.append((measure, measure, rho))
    plt.show()
    for score in scores:
        print(score[0], score[1], '%.2f' % score[2])

def all_plots(refs):
    scores = []
    print('-------------------------------')
    for prev_measure in ['e2e_acc', 'holdout_acc', 'rho']:
        plt.figure(figsize=(20, 3.5))
        for i, next_measure in enumerate(['e2e_acc', 'holdout_acc', 'rho']):
            plt.subplot(1, 4, i + 1)
            rho = run_graphs(script='ilm_rnn2018janb', refs=refs,
                       prev_measure=prev_measure, next_measure=next_measure, add_linear=False)
            scores.append((prev_measure, next_measure, rho))
        plt.show()
    print('-------------------------------')
    print('')
    for score in scores:
        print(score[0], score[1], '%.2f' % score[2])

refs = ['ilmc141']
for ref in refs:
    diag_plots([ref])

# print('---------------')
# for ref in refs:
#     all_plots([ref])
